# Cantera Tutorial: Python

## Getting Started

Open a new Jupyter Notebook. If unfamiliar with Jupyter Notebooks see the [Jupyter Documentation](https://jupyter.readthedocs.io/en/latest/install.html) for installation and basic use instructions. Import the Cantera Python module and NumPy by running:

In [1]:
import cantera as ct
import numpy as np

print(f"Using Cantera version: {ct.__version__}")

Using Cantera version: 2.6.0a4


When using Cantera, the first thing you usually need is an object representing some phase of matter. Here, we'll create a gas mixture:

In [2]:
gas1 = ct.Solution("gri30.yaml")

To view the state of the mixture, *call* the `gas1` object as if it were a function:

In [3]:
gas1()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   0.081894 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy             26469             53361  J
   internal energy       -1.2108e+06        -2.441e+06  J
           entropy             64910        1.3086e+05  J/K
    Gibbs function       -1.9447e+07       -3.9204e+07  J
 heat capacity c_p             14311             28851  J/K
 heat capacity c_v             10187             20536  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -15.717
     [  +52 minor]                 0                 0  



What you have just done is created an object `gas1` that implements GRI-Mech 3.0, the 53-species, 325-reaction natural gas combustion mechanism developed by Gregory P. Smith, David M. Golden, Michael Frenklach, Nigel W. Moriarty, Boris Eiteneer, Mikhail Goldenberg, C. Thomas Bowman, Ronald K. Hanson, Soonho Song, William C. Gardiner, Jr., Vitali V. Lissianski, and Zhiwei Qin.  See the [GRI-Mech Home Page](http://combustion.berkeley.edu/gri-mech/) for more information.

The `gas1` object has properties you would expect for a gas mixture: a temperature, a pressure, species mole and mass fractions, etc. As we will soon see, it has many more properties.

The summary of the state of `gas1` that you found above shows that the new objects created from the `gri30.yaml` input file start out with a temperature of 300 K, a pressure of 1 atm, and have a composition that consists of only one species, in this case hydrogen. There is nothing special about H2 - it just happens to be the first species listed in the input file defining GRI-Mech 3.0. In general, whichever species is listed first will initially have a mole fraction of 1.0, and all others will be zero.

## Setting the State

The state of the object can easily be changed. For example:

In [4]:
gas1.TP = 1200, 101325

sets the temperature to 1200 K and the pressure to 101325 Pa (Cantera always uses SI Units). After this statement, calling `gas1()` results in:

In [5]:
gas1()


  gri30:

       temperature   1200 K
          pressure   1.0133e+05 Pa
           density   0.020473 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        1.3295e+07        2.6802e+07  J
   internal energy        8.3457e+06        1.6825e+07  J
           entropy             85222        1.7181e+05  J/K
    Gibbs function       -8.8972e+07       -1.7937e+08  J
 heat capacity c_p             15377             31000  J/K
 heat capacity c_v             11253             22686  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -17.978
     [  +52 minor]                 0                 0  



Notice that the temperature has been changed as requested, but the density has changed too. The pressure and the composition have not.

Thermodynamics generally requires that *two* properties in addition to composition information be specified to fix the intensive state of a substance (or mixture). The state of the mixture can be set using several combinations of two properties. The following are all equivalent:

In [6]:
gas1.TP = 1200, 101325  # temperature, pressure
gas1.TD = 1200, 0.0204723  # temperature, density
gas1.HP = 1.32956e7, 101325  # specific enthalpy, pressure
gas1.UV = 8.34619e6, 1 / 0.0204723  # specific internal energy, specific volume
gas1.SP = 85227.6, 101325  # specific entropy, pressure
gas1.SV = 85227.6, 1 / 0.0204723  # specific entropy, specific volume

In each case, the values of the extensive properties must be entered *per unit mass*.

Properties may be read independently, such as

In [7]:
gas1.T

1200.5188172713504

or

In [8]:
gas1.h

13302755.250164837

or together:

In [9]:
gas1.UV

(8351530.632807602, 48.84649013545132)

The composition can be set in terms of either mole fractions (`X`) or mass fractions (`Y`):

In [10]:
gas1.X = "CH4:1, O2:2, N2:7.52"

Mass and mole fractions can also be set using the `dict` object, for cases where the composition is stored in a variable or being computed:

In [11]:
phi = 0.8
gas1.X = {"CH4": 1, "O2": 2 / phi, "N2": 2 * 3.76 / phi}

When the composition alone is changed, the temperature and density are held constant. This means that the pressure and other intensive properties will change. The composition can also be set in conjunction with the intensive properties of the mixture:

In [12]:
gas1.TPX = 1200, 101325, "CH4:1, O2:2, N2:7.52"
gas1()


  gri30:

       temperature   1200 K
          pressure   1.0133e+05 Pa
           density   0.28063 kg/m^3
  mean mol. weight   27.633 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        8.6193e+05        2.3818e+07  J
   internal energy        5.0087e+05        1.3841e+07  J
           entropy            8914.2        2.4633e+05  J/K
    Gibbs function       -9.8351e+06       -2.7178e+08  J
 heat capacity c_p            1397.3             38611  J/K
 heat capacity c_v            1096.4             30296  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                O2           0.22014           0.19011           -28.747
               CH4          0.055187          0.095057           -35.961
                N2           0.72467           0.71483           -25.67

The composition above was specified using a string. The format is a comma-separated list of `<species name>:<relative mole numbers>` pairs. The mole numbers will be normalized to produce the mole fractions, and therefore they are "relative" mole numbers. Mass fractions can be set this way too by changing `X` to `Y` in the above statements.

The composition can also be set using an array, which must have the same size as the number of species. For example, to set all 53 mole fractions to the same value, do this:

In [13]:
gas1.X = np.ones(53)  # NumPy array of 53 ones

Or, to set all the mass fractions to equal values:

In [14]:
gas1.Y = np.ones(53)

When setting the state, you can control what properties are held constant by passing the special value `None` to the property setter. For example, to change the specific volume to 2.1 m<sup>3</sup>/kg while holding entropy constant:

In [15]:
gas1.SV = None, 2.1

Or to set the mass fractions while holding temperature and pressure constant:

In [16]:
gas1.TPX = None, None, "CH4:1.0, O2:0.5"

## Working with a Subset of Species

Many properties of a [`Solution`](https://cantera.org/documentation/dev/sphinx/html/cython/importing.html#cantera.Solution) provide values for each species present in the phase. If you want to get values only for a subset of these species, you can use Python's "slicing" syntax to select data for just the species of interest. To get the mole fractions of just the major species in `gas1`, in the order specified, you can write:

In [17]:
Xmajor = gas1["CH4", "O2", "CO2", "H2O", "N2"].X

If you want to use the same set of species repeatedly, you can keep a reference to the sliced phase object:

In [18]:
major = gas1["CH4", "O2", "CO2", "H2O", "N2"]
cp_major = major.partial_molar_cp
wdot_major = major.net_production_rates

The slice object and the original object share the same internal state, so modifications to one will affect the other.

## Working with Mechanism Files

In the previous example, we created an object that models an ideal gas mixture with the species and reactions of GRI-Mech 3.0, using the `gri30.yaml` input file included with Cantera. Several reaction mechanism files in this format are included with Cantera, including ones that model high-temperature air, a hydrogen/oxygen reaction mechanism, and a few surface reaction mechanisms. These files are usually located in the `data` subdirectory of the Cantera installation directory, e.g. `C:\\Program Files\\Cantera\\data` on Windows or `/usr/local/cantera/data/` on Unix/Linux/Mac OS X machines, depending on how you installed Cantera and the options you specified.

If, for some reason, Cantera has difficulty finding where these files are on your system, set environment variable `CANTERA_DATA` to the directory or directories (separated using `;` on Windows or `:` on other operating systems) where they are located. Alternatively, you can call function [`add_directory`](https://cantera.org/documentation/dev/sphinx/html/cython/importing.html#cantera.add_directory) to add a directory to the Cantera search path:

In [19]:
ct.add_directory("/usr/local/cantera/my_data_files")

Cantera input files are plain text files, and can be created with any text editor. See the guide *[Defining Phases](https://cantera.org/tutorials/yaml/defining-phases.html)* for more information.

A Cantera input file may contain more than one phase specification, or may contain specifications of interfaces (surfaces). Here, we import definitions of two bulk phases and the interface between them from the file `diamond.yaml`:

In [20]:
gas2 = ct.Solution("diamond.yaml", "gas")
diamond = ct.Solution("diamond.yaml", "diamond")
diamond_surf = ct.Interface("diamond.yaml", "diamond_100", [gas2, diamond])

Note that the bulk (i.e., 3D or homogenous) phases that participate in the surface reactions must also be passed as arguments to [`Interface`](https://cantera.org/documentation/dev/sphinx/html/cython/importing.html#cantera.Interface).

### Converting CK-format files

See *[Converting CK-format files](https://cantera.org/tutorials/ck2yaml-tutorial.html)* in the *[Working with Input Files](https://cantera.org/tutorials/input-files.html)* documentation.

## Getting Help

In addition to the *[Python Module Documentation](https://cantera.org/documentation/index.html)*, documentation of the Python classes and their methods can be accessed from within the Python interpreter as well.

Suppose you have created a Cantera object and want to know what methads are avialable for it, and get help on using the methods:

In [21]:
g = ct.Solution("gri30.yaml")

To get help on the Python class that this object is an instance of, put a question mark `?` after the variable:

In [22]:
?g

Call signature: g(*args, **kwargs)
Type:           Solution
String form:    <cantera.composite.Solution object at 0xffff81f4eac0>
File:           /opt/conda/lib/python3.9/site-packages/cantera/composite.py
Docstring:     
A class for chemically-reacting solutions. Instances can be created to
represent any type of solution -- a mixture of gases, a liquid solution, or
a solid solution, for example.

Class `Solution` derives from classes `ThermoPhase`, `Kinetics`, and
`Transport`.  It defines no methods of its own, and is provided so that a
single object can be used to compute thermodynamic, kinetic, and transport
properties of a solution.

To skip initialization of the Transport object, pass the keyword argument
``transport_model=None`` to the `Solution` constructor.

The most common way to instantiate `Solution` objects is by using a phase
definition, species and reactions defined in an input file::

    gas = ct.Solution('gri30.yaml')

If an input file defines multiple phases, the corr

For a simple list of the properties and methods of this object:

In [23]:
dir(g)

['CK_mode',
 'DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'Te',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__composition_to_array',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_cython__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__setstate_cython__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_cinit',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_init_yaml',
 '_native_state',
 '_partial_states',
 '_references',
 'activities',
 'activity_coefficients'

To get help on a specific method, e.g. the `species_index` method:

In [24]:
?g.species_index

Docstring:
ThermoPhase.species_index(self, species) -> int

The index of species *species*, which may be specified as a string or
an integer. In the latter case, the index is checked for validity and
returned. If no such species is present, an exception is thrown.
Type:      builtin_function_or_method


For properties, getting the documentation is slightly trickier, as the usual method will give you help for the *result*, e.g.:

In [25]:
?g.T

Type:        float
String form: 300.0
Docstring:   Convert a string or number to a floating point number, if possible.


provides help on Python's `float` class. To get the help for the temperature property, ask for the attribute of the class object itself:

In [26]:
?g.__class__.T

Type:        getset_descriptor
String form: <attribute 'T' of 'cantera._cantera.ThermoPhase' objects>
Docstring:   Temperature [K].


Help can also be obtained using the `help` function:

In [27]:
help(g.species_index)

Help on built-in function species_index:

species_index(...) method of cantera.composite.Solution instance
    ThermoPhase.species_index(self, species) -> int
    
    The index of species *species*, which may be specified as a string or
    an integer. In the latter case, the index is checked for validity and
    returned. If no such species is present, an exception is thrown.



## Chemical Equilibrium

To set a gas mixture to a state of chemical equilibrium, use the `equilibrate` method:

In [28]:
g = ct.Solution("gri30.yaml")
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("TP")

The above statement sets the state of object `g` to the state of chemical equilibrium holding temperature and pressure fixed. Alternatively, the specific enthalpy and pressure can be held fixed:

In [29]:
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("HP")

Other options are:
* `'UV'` for fixed specific internal energy and specific volume
* `'SV'` for fixed specific entropy and specific volume
* `'SP'` for fixed specific entropy and pressure

How can you tell if `equilibrate` has correctly found the chemical equilibrium state? One way is to verify that the net rates of progress of all reversible reactions are zero. Here is the code to do this:

In [30]:
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("HP")

In [31]:
rf = g.forward_rates_of_progress
rr = g.reverse_rates_of_progress
for i in range(g.n_reactions):
    if g.is_reversible(i) and rf[i] != 0.0:
        print("  %4i   %10.4g  " % (i, (rf[i] - rr[i]) / rf[i]))

     0    3.523e-16  
     1    1.851e-16  
     2    5.904e-16  
     3   -5.209e-15  
     4   -5.111e-15  
     5    1.052e-14  
     6   -3.396e-15  
     7    5.229e-15  
     8     3.62e-15  
     9   -1.904e-15  
    10   -3.274e-16  
    11    1.891e-15  
    12    4.946e-15  
    13    3.011e-15  
    14    2.147e-16  
    15   -6.937e-15  
    16   -1.238e-14  
    17   -4.378e-15  
    18   -1.142e-14  
    19    5.548e-15  
    20   -8.799e-16  
    21    3.269e-15  
    22    8.803e-15  
    23    1.036e-14  
    24    2.055e-15  
    25    5.512e-15  
    26   -9.727e-15  
    27    3.085e-15  
    28   -1.346e-14  
    29    3.681e-15  
    30   -2.105e-15  
    31   -9.781e-16  
    32    5.225e-15  
    33    5.269e-15  
    34    5.073e-15  
    35    5.276e-15  
    37    4.428e-15  
    38   -1.423e-15  
    39   -1.372e-15  
    40   -1.346e-15  
    41   -1.215e-15  
    42    1.916e-15  
    43   -5.255e-15  
    44   -4.762e-15  
    45   -6.832e-15  
    46   -

If the magnitudes of the numbers in this list are all very small (which in this case they are), then each reversible reaction is very nearly equilibrated, which only occurs if the gas is in chemical equilibrium.

You might be wondering how `equilibrate` works. (Then again, you might not.) Method `equilibrate` invokes Cantera's chemical equilibrium solver, which uses an element potential method. The element potential method is one of a class of equivalent *nonstoichiometric* methods that all have the characteristic that the probelm reduces to solving a set of $M$ nonlinear algebraic equations, where $M$ is the number of elements (not species). The so-called *stoichiometric* methods, on the other hand (including the Gibbs minimization), require solving $K$ nonlinear equations, where $K$ is the number of species (usually $K >> M$). See Smith and Missen's "Chemical Reaction Equilibrium Analysis" for more information on the various algorithms and their characteristics.

Cantera uses a damped Newton method to solve these equations, and does a few other things to generate a good starting guess and to produce a reasonably robust algorithm. If you want to know more about the details, look at the on-line documentated source code of Cantera C++ class [`ChemEquil`](https://cantera.org/documentation/dev/doxygen/html/da/d0e/classCantera_1_1ChemEquil.html).

## Chemical Kinetics

[`Solution`](https://cantera.org/documentation/dev/sphinx/html/cython/importing.html#cantera.Solution) objects are also [`Kinetics`](https://cantera.org/documentation/dev/sphinx/html/cython/kinetics.html#cantera.Kinetics) objects, and provide all of the methods necessary to compute the thermodynamic quantities associated with each reaction, reaction rates, and species creation and destruction rates. They also provide methods to inspect the quantities that define each reaction, such as the rate constants and the stoichiometric coefficients. The rate calculation functions are used extensively within Cantera's *[reactor network model](https://cantera.org/documentation/dev/sphinx/html/cython/zerodim.html#sec-cython-zerodim)* and *[1D flame model](https://cantera.org/documentation/dev/sphinx/html/cython/onedim.html#sec-cython-onedim)*.

Information about individual reactions that are independent of the thermodynamic state can be obtained by accessing [`Reaction`](https://cantera.org/documentation/dev/sphinx/html/cython/kinetics.html#cantera.Reaction) objects with the [`Kinetics.reaction`](https://cantera.org/documentation/dev/sphinx/html/cython/kinetics.html#cantera.Kinetics.reaction) method:

In [32]:
g = ct.Solution("gri30.yaml")
r = g.reaction(2)  # get a Reaction object
r

<ElementaryReaction: H2 + O <=> H + OH>

In [33]:
r.reactants

{'H2': 1.0, 'O': 1.0}

In [34]:
r.products

{'H': 1.0, 'OH': 1.0}

In [35]:
r.rate

<ArrheniusRate at ffff81ea1c60>

If we are interested in only certain types of reactions, we can use this information to filter the full list of reactions to find just the ones of interest. For example, here we find the indices of just those reactions which convert `CO` into `CO2`:

In [36]:
II = [
    i
    for i, r in enumerate(g.reactions())
    if "CO" in r.reactants and "CO2" in r.products
]

for i in II:
    print(g.reaction(i).equation)

CO + O (+M) <=> CO2 (+M)
CO + O2 <=> CO2 + O
CO + OH <=> CO2 + H
CO + HO2 <=> CO2 + OH


(Actually, we should also include the reactions where the reaction is written such that `CO2` is a reactant and `CO` is a product, but for this example, we'll just stick to this smaller set of reactions.) Now, let's set the composition to an interesting state:

In [37]:
g.TPX = 300, 101325, {"CH4": 0.6, "O2": 1.0, "N2": 3.76}
g.equilibrate("HP")

We can verify that this is an equilibrium state by seeing that the net reactions rates are essentially zero:

In [38]:
g.net_rates_of_progress[II]

array([-2.64274280e-19, -6.56450534e-21,  0.00000000e+00, -9.82558219e-20])

Now let's see what happens if we decrease the temperature of the mixture:

In [39]:
g.TP = g.T - 100, None
g.net_rates_of_progress[II]

array([3.18644907e-05, 5.00489883e-08, 1.05964910e-01, 2.89502678e-06])

All of the reaction rates are positive, favoring the formation of `CO2` from `CO`, with the third reaction, `CO + OH <=> CO2 + H` proceeding the fastest. If we look at the enthalpy change associated with each of these reactions:

In [40]:
g.delta_enthalpy[II]

array([-5.33034690e+08, -2.23248529e+07, -8.76650141e+07, -2.49169644e+08])

we see that the change is negative in each case, indicating a net release of thermal energy. The total heat release rate can be computed either from the reaction rates:

In [41]:
np.dot(g.net_rates_of_progress, g.delta_enthalpy)

-58013369.59815948

or from the species production rates:

In [42]:
np.dot(g.net_production_rates, g.partial_molar_enthalpies)

-58013369.598159574

The contribution from just the selected reactions is:

In [43]:
np.dot(g.net_rates_of_progress[II], g.delta_enthalpy[II])

-9307122.692794016

or about 16% of the total heat release rate.